In [182]:
import time
import os
import pandas as pd
import numpy as np
import xgboost as xgb

from datetime import datetime

** Read Data **

In [183]:
train = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Train.csv')
test = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Test_D7W1juQ.csv')
sample_submission = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\sample_submission_cvwryvM.csv')
health_camp_detail = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Health_Camp_Detail.csv')
patient_profile = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Patient_Profile.csv')
first_health_camp_attended = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\First_Health_Camp_Attended.csv')
second_health_camp_attended = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Second_Health_Camp_Attended.csv')
third_health_camp_attended = pd.read_csv('D:\\Data Science\\AV\\Knocktober\\data\\Train\\Third_Health_Camp_Attended.csv')
print "train", train.shape
print "test", test.shape
print "health_camp_detail", health_camp_detail.shape
print "patient_profile", patient_profile.shape
print "first_health_camp_attended", first_health_camp_attended.shape
print "second_health_camp_attended", second_health_camp_attended.shape
print "third_health_camp_attended", third_health_camp_attended.shape

train (75278, 8)
test (35249, 8)
health_camp_detail (65, 6)
patient_profile (37633, 11)
first_health_camp_attended (6218, 5)
second_health_camp_attended (7819, 3)
third_health_camp_attended (6515, 4)


** Data Glimpse **

In [184]:
train.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0


In [185]:
test.head(3)

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,02-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0


In [189]:
health_camp_detail.head(3)

,Health_Camp_ID,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3
0,6560,16-Aug-03,20-Aug-03,First,B,2
1,6530,16-Aug-03,28-Oct-03,First,C,2
2,6544,03-Nov-03,15-Nov-03,First,F,1


In [190]:
first_health_camp_attended.head(3)

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN


In [191]:
second_health_camp_attended.head(3)

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181


In [192]:
third_health_camp_attended.head(3)

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1


In [193]:
patient_profile.head(3)

,Patient_ID,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,516956,0,0,0,0,1,90,39,18-Jun-03,NaN,Software Industry
1,507733,0,0,0,0,1,None,40,20-Jul-03,H,Software Industry
2,508307,0,0,0,0,3,87,46,02-Nov-02,D,BFSI


** Data Preparation & Checking **

In [194]:
def create_combined_df(train, test):
    '''
    Creates combined dataframe from train & test dataframes.
    Basically concats train & test dataframes.
    
    Parameters
    -----------
    train : a pandas DataFrame.
    test : a pandas DataFrame.
    
    Returns
    --------
    combined_df : concated pandas dataframe.
    '''
    train['train_or_test'] = 'train'
    test['train_or_test'] = 'test'
    combined_df = pd.concat([train,test])
    return combined_df

In [195]:
# Combining Train & Test
combined_df = create_combined_df(train, test)
print combined_df.shape
combined_df.head(2)

(110527, 9)


,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,train_or_test
0,489652,6578,10-Sep-05,4,0,0,0,2,train
1,507246,6578,18-Aug-05,45,5,0,0,7,train


In [196]:
# Registration_Date is missing only in train
print combined_df[combined_df.Registration_Date.isnull()]['train_or_test'].unique()
print combined_df[combined_df.isnull().any(axis=1)]['train_or_test'].unique()

['train']
['train']


In [197]:
def extract_from_date(date):
    date_fields = []
    if type(date) is np.float:
        date_fields = [-999,-999,-999] 
    else:
        datetime_object = datetime.strptime(date, '%d-%b-%y')
        date_fields = [datetime_object.day, datetime_object.month, datetime_object.year]
    return date_fields

In [198]:
combined_df['Reg_Date'] = combined_df['Registration_Date'].apply(extract_from_date)
combined_df['Reg_Day'] = combined_df['Reg_Date'].apply(lambda x: x[0])
combined_df['Reg_Month'] = combined_df['Reg_Date'].apply(lambda x: x[1])
combined_df['Reg_Year'] = combined_df['Reg_Date'].apply(lambda x: x[2])
combined_df.drop(['Registration_Date','Reg_Date'], axis=1, inplace=True)
combined_df.head(2)

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,train_or_test,Reg_Day,Reg_Month,Reg_Year
0,489652,6578,4,0,0,0,2,train,10,9,2005
1,507246,6578,45,5,0,0,7,train,18,8,2005


** Generating the outcome **

In [199]:
col_names = [['Patient_ID','Health_Camp_ID','Outcome']]
first_health_camp_attended = first_health_camp_attended[['Patient_ID','Health_Camp_ID','Health_Score']]
first_health_camp_attended.columns = col_names
first_health_camp_attended.head(2)

,Patient_ID,Health_Camp_ID,Outcome
0,506181,6560,0.439024
1,494977,6560,0.097561


In [200]:
second_health_camp_attended = second_health_camp_attended[['Patient_ID','Health_Camp_ID','Health Score']]
second_health_camp_attended.columns = col_names
second_health_camp_attended.head(2)

,Patient_ID,Health_Camp_ID,Outcome
0,526631,6536,0.875136
1,509122,6536,0.755700


In [201]:
third_health_camp_attended = third_health_camp_attended[['Patient_ID','Health_Camp_ID','Number_of_stall_visited']]
print third_health_camp_attended.shape
third_health_camp_attended = third_health_camp_attended[third_health_camp_attended['Number_of_stall_visited']>0]
third_health_camp_attended.columns = col_names
print third_health_camp_attended.shape

(6515, 3)
(6497, 3)


In [202]:
all_camps = pd.concat([first_health_camp_attended, second_health_camp_attended, third_health_camp_attended])
all_camps['Outcome'] = 1
print all_camps.shape

(20534, 3)


In [203]:
all_camps.head(2)

,Patient_ID,Health_Camp_ID,Outcome
0,506181,6560,1
1,494977,6560,1


In [204]:
combined_df = combined_df.merge(all_camps, on=['Patient_ID','Health_Camp_ID'], how='left')
combined_df.head(2)

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,train_or_test,Reg_Day,Reg_Month,Reg_Year,Outcome
0,489652,6578,4,0,0,0,2,train,10,9,2005,1.0
1,507246,6578,45,5,0,0,7,train,18,8,2005,NaN


In [205]:
combined_df['Outcome'] = combined_df['Outcome'].fillna(0).astype('int')
combined_df.head(3)

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,train_or_test,Reg_Day,Reg_Month,Reg_Year,Outcome
0,489652,6578,4,0,0,0,2,train,10,9,2005,1
1,507246,6578,45,5,0,0,7,train,18,8,2005,0
2,523729,6534,0,0,0,0,0,train,29,4,2006,1


In [206]:
print combined_df[combined_df['train_or_test']=='train'].Outcome.value_counts()
print combined_df.train_or_test.value_counts()

0    54744
1    20534
Name: Outcome, dtype: int64
train    75278
test     35249
Name: train_or_test, dtype: int64


i.e. ~27% of people who register for the camp, turns up!!

# XGB Model

In [207]:
train = combined_df[combined_df['train_or_test']=='train']
test = combined_df[combined_df['train_or_test']=='test']
print train.shape
print test.shape

(75278, 12)
(35249, 12)


In [208]:
y_train = train.Outcome.values
train.drop(['train_or_test','Patient_ID','Health_Camp_ID','Outcome'], axis=1, inplace=True)
sub_df = test[['Patient_ID','Health_Camp_ID']]
test.drop(['train_or_test','Patient_ID','Health_Camp_ID','Outcome'], axis=1, inplace=True)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [209]:
print y_train
print sub_df.head(2)
print train.head(2)
print test.head(2)

[1 0 1 ..., 1 0 0]
       Patient_ID  Health_Camp_ID
75278      505701            6548
75279      500633            6584
   Var1  Var2  Var3  Var4  Var5  Reg_Day  Reg_Month  Reg_Year
0     4     0     0     0     2       10          9      2005
1    45     5     0     0     7       18          8      2005
       Var1  Var2  Var3  Var4  Var5  Reg_Day  Reg_Month  Reg_Year
75278     1     0     0     0     2       21          5      2006
75279     0     0     0     0     0        2          6      2006


In [210]:
dtrain = xgb.DMatrix(train.values, label=y_train)
print dtrain.num_row()
print dtrain.num_col()

75278
8


In [211]:
dtest = xgb.DMatrix(test.values)
print dtest.num_row()
print dtest.num_col()

35249
8


In [212]:
# XGB Params
param = {'max_depth':10, 'eta':0.1, 'silent':1, 'objective':'binary:logistic' }
#param['nthread'] = 7
param['eval_metric'] = 'auc'
param['subsample'] = 0.9
param['colsample_bytree']= 0.7
#param['min_child_weight'] = 0
param['booster'] = "gbtree"
param['gamma'] = 0.005
param['seed'] = 1729

In [213]:
watchlist = [(dtrain,'train')]
num_round = 500
early_stopping_rounds = 35

** Cross Validation **

In [216]:
t0 = time.time()
xgb_cv = xgb.cv(param, 
                dtrain, 
                num_boost_round=num_round, 
                nfold=10, 
                #stratified=True, 
                early_stopping_rounds=early_stopping_rounds, 
                maximize=True,
                seed=1729, 
                verbose_eval=50,
                show_stdv=True)
print time_elapsed(t0), "minutes elapsed!"
xgb_cv

[0]	train-auc:0.689072+0.000827846	test-auc:0.672135+0.00787646
[50]	train-auc:0.758637+0.00161371	test-auc:0.72959+0.00705422
0.721333332856 minutes elapsed!


,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.672135,0.007876,0.689072,0.000828
1,0.685304,0.014381,0.700635,0.016455
2,0.699028,0.012588,0.713773,0.011881
3,0.701820,0.014323,0.717172,0.010523
4,0.707353,0.013595,0.722555,0.008570
5,0.712857,0.009861,0.728165,0.005107
6,0.712467,0.008739,0.728259,0.003884
7,0.714642,0.010471,0.730585,0.005332
8,0.716024,0.010937,0.732405,0.005432
9,0.716479,0.010149,0.733337,0.005139


** Training **

In [217]:
t0 = time.time()
num_train_round = xgb_cv.shape[0]
print num_train_round
xgbModel = xgb.train(param, 
                     dtrain, 
                     num_boost_round=num_train_round,
                     evals=watchlist, 
                     maximize=True, 
                     verbose_eval=100)
print time_elapsed(t0), "minutes elapsed!"

62
[0]	train-auc:0.569114
0.0492833336194 minutes elapsed!


** Feature Importance **

In [218]:
def importance_XGB(xgbModel, cols_list):
    impdf = []
    for ft, score in xgbModel.get_fscore().iteritems():
        impdf.append({'feature': ft, 'feature_name': cols_list[int(ft[1:])], 'importance': score})
    impdf = pd.DataFrame(impdf)
    impdf = impdf.sort_values(by='importance', ascending=False).reset_index(drop=True)
    impdf['importance'] /= impdf['importance'].sum()
    return impdf

In [219]:
impdf = importance_XGB(xgbModel, train.columns)
impdf

,feature,feature_name,importance
0,f5,Reg_Day,0.405264
1,f0,Var1,0.186622
2,f4,Var5,0.137563
3,f6,Reg_Month,0.112570
4,f1,Var2,0.089300
5,f3,Var4,0.034474
6,f7,Reg_Year,0.034010
7,f2,Var3,0.000199


** Prediction **

In [178]:
pred = xgbModel.predict(dtest)
pred

array([ 0.22707318,  0.15854152,  0.23217027, ...,  0.22660133,
        0.24474473,  0.46652943], dtype=float32)

In [179]:
sub_df['Outcome'] = pred
print sub_df.shape
sub_df.head(3)

(35249, 3)


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Patient_ID,Health_Camp_ID,Outcome
75278,505701,6548,0.227073
75279,500633,6584,0.158542
75280,506945,6582,0.232170


In [180]:
# Write csv pred file
sub_df.to_csv('D:\\Data Science\\AV\\Knocktober\\submissions\\benchmark_only_train_v2.csv', index=False)

# Utils

In [128]:
def time_elapsed(t0):
    return (time.time()-t0)/60